# 图生图-ControlNet-Canny

同济子豪兄 2023-6-13

MMagic文档：https://github.com/open-mmlab/mmagic/tree/main/configs/controlnet

ControlNet官方Repo：https://github.com/lllyasviel/ControlNet

## 进入 MMagic 主目录

In [1]:
import os
os.chdir('../../mmagic')

In [2]:
# !conda install -c conda-forge ipywidgets -y

## 导入工具包

In [3]:
import cv2
import numpy as np
import mmcv
from mmengine import Config
from PIL import Image

from mmagic.registry import MODELS
from mmagic.utils import register_all_modules

register_all_modules()

/root/miniconda3/envs/mmagic2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# !pip install accelerate
# # !conda install xformers -c  xformers -y
# !pip install -U xformers

In [5]:
# !python -m xformers.info|

In [6]:
# !pip install -U xformers

## 载入ControlNet模型

In [7]:
cfg = Config.fromfile('configs/controlnet/controlnet-canny.py')
controlnet = MODELS.build(cfg.model).cuda()

06/19 02:58:17 - mmengine - INFO - Set model dtype to 'torch.float32'.
06/19 02:58:20 - mmengine - INFO - Creating runwayml/stable-diffusion-v1-5 by 'HuggingFace'


/mmagic/mmagic/models/archs/wrapper.py:149: FutureWarning: Accessing config attribute `block_out_channels` directly via 'AutoencoderKL' object attribute is deprecated. Please access 'block_out_channels' over 'AutoencoderKL's config object instead, e.g. 'unet.config.block_out_channels'.
  return getattr(self.model, name)


06/19 02:58:23 - mmengine - INFO - Enable Xformers for HuggingFace Diffusers' module 'vae'.
06/19 02:58:23 - mmengine - INFO - Enable Xformers for HuggingFace Diffusers' module 'unet'.
06/19 02:58:23 - mmengine - INFO - Set model dtype to 'torch.float32'.
06/19 02:58:23 - mmengine - INFO - Enable Xformers for HuggingFace Diffusers' module 'model'.


## 输入Canny边缘图

In [18]:
control_url = '/MMagic_Tutorials/0614/origin_image.jpg'
control_img = mmcv.imread(control_url)

# Apply Canny edge detection
control = cv2.Canny(control_img, 100, 200)

# Expand dimensions to make it a 3 channel image
control = control[:, :, None]
control = np.concatenate([control] * 3, axis=2)

# Now control is a numpy array, you can check it and then save it
if isinstance(control, np.ndarray):
    cv2.imwrite('canny_image.jpg', control)
else:
    print('Image processing failed')


## 咒语Prompt

In [19]:
prompt = 'Chinese-style decoration.'

## 执行预测

In [ ]:
output_dict = controlnet.infer(prompt, control=control)
samples = output_dict['samples']
for idx, sample in enumerate(samples):
    sample.save(f'sample_{idx}.png')
controls = output_dict['controls']
for idx, control in enumerate(controls):
    control.save(f'control_{idx}.png')